## Import necessary libraries

In [1]:
import pandas as pd
import requests

## get data from APIs (used tmdb api)

In [14]:
df = pd.DataFrame()

In [37]:
l = []
for i in range(1,472):
    response = requests.get(f'https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={i}')
    temp = pd.DataFrame(response.json()['results'])[['title','overview','genre_ids']]
    l.append(temp)
df = pd.concat(l,ignore_index=True)

In [38]:
df

,title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,12 Angry Men,The defense and the prosecution have rested an...,[18]
...,...,...,...
9415,A Fantastic Fear of Everything,Jack is a children's author turned crime novel...,"[35, 53, 27]"
9416,The House Bunny,Shelley is living a carefree life until a riva...,"[10749, 35]"
9417,Freaks - You're One of Us,After having a chance encounter with a mysteri...,"[18, 14, 53]"
9418,Pathfinder,A Viking boy is left behind after his clan bat...,"[12, 28]"


In [39]:
df.head()

,title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,12 Angry Men,The defense and the prosecution have rested an...,[18]


In [24]:
genre_api = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US')

In [28]:
movie_genre = pd.DataFrame(genre_api.json()['genres'])

In [29]:
movie_genre

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


## Creating a mapping dictionary

In [86]:
movie_dict = {}
for i,j in zip(movie_genre['id'],movie_genre['name']):
    movie_dict[i]=j

In [87]:
movie_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

## Changing ids to genres

In [82]:
for i in df['genre_ids']:
    for j in range(0,len(i)):
        i[j] = movie_dict[i[j]]

In [83]:
df['genre_ids']

0                           [Drama, Crime]
1                           [Drama, Crime]
2                           [Drama, Crime]
3                    [Drama, History, War]
4                                  [Drama]
                       ...                
9415            [Comedy, Thriller, Horror]
9416                     [Romance, Comedy]
9417            [Drama, Fantasy, Thriller]
9418                   [Adventure, Action]
9419    [Horror, Fantasy, Comedy, Mystery]
Name: genre_ids, Length: 9420, dtype: object

In [84]:
df.head()

,title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,12 Angry Men,The defense and the prosecution have rested an...,[Drama]


## Text preprocessing

**text normalization**

In [104]:
for i in range(0,len(df['overview'])):
    df['overview'][i] = df['overview'][i].lower()

In [107]:
df.head(10)

,title,overview,genre_ids
0,The Shawshank Redemption,imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,The Godfather,"spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,The Godfather Part II,in the continuing saga of the corleone crime f...,"[Drama, Crime]"
3,Schindler's List,the true story of how businessman oskar schind...,"[Drama, History, War]"
4,12 Angry Men,the defense and the prosecution have rested an...,[Drama]
5,Spirited Away,"a young girl, chihiro, becomes trapped in a st...","[Animation, Family, Fantasy]"
6,The Dark Knight,batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]"
7,Dilwale Dulhania Le Jayenge,"raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
8,The Green Mile,a supernatural tale set on death row in a sout...,"[Fantasy, Drama, Crime]"
9,Parasite,"all unemployed, ki-taek's family takes peculia...","[Comedy, Thriller, Drama]"


**punctuation removal**

In [109]:
import string
exclude = string.punctuation

In [114]:
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

In [115]:
df['overview'] = df['overview'].apply(remove_punc)

In [116]:
df['overview'][0]

'imprisoned in the 1940s for the double murder of his wife and her lover upstanding banker andy dufresne begins a new life at the shawshank prison where he puts his accounting skills to work for an amoral warden during his long stretch in prison dufresne comes to be admired by the other inmates  including an older prisoner named red  for his integrity and unquenchable sense of hope'

In [124]:
!pip install nltk

In [127]:
from nltk.corpus import stopwords

In [134]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [135]:
def remove_stopword(text):
    new_text = []
    for i in text.split():
        if i not in stopwords.words('english'):
            new_text.append(i)
    return " ".join(new_text)

In [136]:
df['overview'] = df['overview'].apply(remove_stopword)

In [137]:
df['overview']

0       imprisoned 1940s double murder wife lover upst...
1       spanning years 1945 1955 chronicle fictional i...
2       continuing saga corleone crime family young vi...
3       true story businessman oskar schindler saved t...
4       defense prosecution rested jury filing jury ro...
                              ...                        
9415    jack childrens author turned crime novelist wh...
9416    shelley living carefree life rival gets tossed...
9417    chance encounter mysterious character wendy yo...
9418    viking boy left behind clan battles native ame...
9419    haunting ghost story set backdrop busy winter ...
Name: overview, Length: 9420, dtype: object

In [141]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [142]:
def to_token(text):
    return word_tokenize(text)

In [143]:
df['overview'] = df['overview'].apply(to_token)

In [144]:
df['overview']

0       [imprisoned, 1940s, double, murder, wife, love...
1       [spanning, years, 1945, 1955, chronicle, ficti...
2       [continuing, saga, corleone, crime, family, yo...
3       [true, story, businessman, oskar, schindler, s...
4       [defense, prosecution, rested, jury, filing, j...
                              ...                        
9415    [jack, childrens, author, turned, crime, novel...
9416    [shelley, living, carefree, life, rival, gets,...
9417    [chance, encounter, mysterious, character, wen...
9418    [viking, boy, left, behind, clan, battles, nat...
9419    [haunting, ghost, story, set, backdrop, busy, ...
Name: overview, Length: 9420, dtype: object

In [145]:
df['overview'][0]

['imprisoned',
 '1940s',
 'double',
 'murder',
 'wife',
 'lover',
 'upstanding',
 'banker',
 'andy',
 'dufresne',
 'begins',
 'new',
 'life',
 'shawshank',
 'prison',
 'puts',
 'accounting',
 'skills',
 'work',
 'amoral',
 'warden',
 'long',
 'stretch',
 'prison',
 'dufresne',
 'comes',
 'admired',
 'inmates',
 'including',
 'older',
 'prisoner',
 'named',
 'red',
 'integrity',
 'unquenchable',
 'sense',
 'hope']

In [146]:
df['stemmed_overview'] = None

In [147]:
df.head()

,title,overview,genre_ids,stemmed_overview
0,The Shawshank Redemption,"[imprisoned, 1940s, double, murder, wife, love...","[Drama, Crime]",None
1,The Godfather,"[spanning, years, 1945, 1955, chronicle, ficti...","[Drama, Crime]",None
2,The Godfather Part II,"[continuing, saga, corleone, crime, family, yo...","[Drama, Crime]",None
3,Schindler's List,"[true, story, businessman, oskar, schindler, s...","[Drama, History, War]",None
4,12 Angry Men,"[defense, prosecution, rested, jury, filing, j...",[Drama],None


In [148]:
from nltk.stem.porter import PorterStemmer

In [151]:
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text])

In [152]:
df['stemmed_overview'] = df['overview'].apply(stem_words)

In [153]:
df.head()

,title,overview,genre_ids,stemmed_overview
0,The Shawshank Redemption,"[imprisoned, 1940s, double, murder, wife, love...","[Drama, Crime]",imprison 1940 doubl murder wife lover upstand ...
1,The Godfather,"[spanning, years, 1945, 1955, chronicle, ficti...","[Drama, Crime]",span year 1945 1955 chronicl fiction italianam...
2,The Godfather Part II,"[continuing, saga, corleone, crime, family, yo...","[Drama, Crime]",continu saga corleon crime famili young vito c...
3,Schindler's List,"[true, story, businessman, oskar, schindler, s...","[Drama, History, War]",true stori businessman oskar schindler save th...
4,12 Angry Men,"[defense, prosecution, rested, jury, filing, j...",[Drama],defens prosecut rest juri file juri room decid...


In [159]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


True

In [160]:
wordnet = WordNetLemmatizer()

In [161]:
def lemme_words(text):
    return " ".join([wordnet.lemmatize(word) for word in text])

In [162]:
df['lemmetized_overview'] = None

In [163]:
df['lemmetized_overview'] = df['overview'].apply(lemme_words)

In [164]:
df.head()

,title,overview,genre_ids,stemmed_overview,lemmetized_overview
0,The Shawshank Redemption,"[imprisoned, 1940s, double, murder, wife, love...","[Drama, Crime]",imprison 1940 doubl murder wife lover upstand ...,imprisoned 1940s double murder wife lover upst...
1,The Godfather,"[spanning, years, 1945, 1955, chronicle, ficti...","[Drama, Crime]",span year 1945 1955 chronicl fiction italianam...,spanning year 1945 1955 chronicle fictional it...
2,The Godfather Part II,"[continuing, saga, corleone, crime, family, yo...","[Drama, Crime]",continu saga corleon crime famili young vito c...,continuing saga corleone crime family young vi...
3,Schindler's List,"[true, story, businessman, oskar, schindler, s...","[Drama, History, War]",true stori businessman oskar schindler save th...,true story businessman oskar schindler saved t...
4,12 Angry Men,"[defense, prosecution, rested, jury, filing, j...",[Drama],defens prosecut rest juri file juri room decid...,defense prosecution rested jury filing jury ro...
